# part1: workflow

In [ ]:
from llama_index.core.workflow import Workflow
from llama_index.core.workflow import step
from llama_index.core.workflow import Event
from llama_index.core.workflow import StartEvent
from llama_index.core.workflow import StopEvent
from llama_index.core.workflow import Context
from llama_index.utils.workflow import draw_all_possible_flows

# event

In [ ]:
class QueryEvent(Event):
    question: str


class AnswerEvent(Event):
    question: str
    answer: str

# workflow

In [ ]:
class SubQuestionQueryEngine(Workflow):
    @step
    async def query(self, ctx: Context, ev: StartEvent) -> QueryEvent:
        # Fake subquestions gen
        FAKE_NUM_SUB_QUESTION = 5
        sub_questions = [f'q{i}' for i in range(FAKE_NUM_SUB_QUESTION)]

        # get num_question
        num_question = len(sub_questions)
        await ctx.store.set("num_question", len(sub_questions))

        for q in sub_questions:
            #self.send_event(QueryEvent(question=question))
            print(f"send: {q}")
            ctx.send_event(QueryEvent(question=q))
        return None

    @step
    async def sub_question(self, ctx: Context, ev: QueryEvent) -> AnswerEvent:
        print(f"Sub-question is {ev.question}")
        # get fake answer
        answer = f"answer of: {ev.question}"
        return AnswerEvent(question=ev.question, answer=answer)

    @step
    async def combine_answers(
        self, ctx: Context, ev: AnswerEvent
    ) -> StopEvent | None:
        num_question = await ctx.store.get("num_question")
        # wait until we receive 3 events
        result = ctx.collect_events(ev, [AnswerEvent] * num_question)
        if result is None:
            print('combine_answers output None')
            return None

        # do something with all {num_question} together
        print(result)
        return StopEvent(result="Done")

In [ ]:
draw_all_possible_flows(
    SubQuestionQueryEngine, filename="fake_sub_question_query_engine.html"
)

In [ ]:
w = SubQuestionQueryEngine(timeout=10, verbose=False)
result = await w.run()
print('---')
print(result)

# part2: sub_question

In [ ]:
qset = {
  "id": "113-1-1-med-surg",
  "year": "113",
  "time": "1",
  "qid": "1",
  "discipline": "內外科護理學",
  "ans": "C",
  "question": "有關多發性硬化症之診斷檢查，下列何者錯誤？",
  "options": {
   "A": "腦脊髓液分析可發現IgG抗體上升",
   "B": "視覺誘發電位可觀察到受損的神經在傳導過程出現延遲和中斷",
   "C": "超音波檢查可發現中樞神經系統髓鞘脫失",
   "D": "核磁共振影像可用來確認多發性硬化症之斑塊"
  },
  "discipline_slug": "med-surg"
}

In [ ]:
question = f"以下為單選題，請從各選項中選擇最符合題意的答案：\n題幹: {qset['question']}.\n選項: \nA: {qset['options']['A']}; B: {qset['options']['B']}; C: {qset['options']['C']}; D: {qset['options']['D']}."
print(question)

In [ ]:
prompt_en = f"""Given a user question, output a list of relevant sub-questions, such that the answers to all the
sub-questions put together will answer the question. 
Respond in pure JSON without any markdown, like this:
{{
    "sub_questions": [
        "What is the population of San Francisco?",
        "What is the budget of San Francisco?",
        "What is the GDP of San Francisco?"
    ]
}}
Here is the user question: {question}

"""

In [ ]:
print(prompt_en)

In [ ]:
prompt = f"""給定一個使用者的問題，請輸出一系列相關的子問題，讓所有子問題的答案合在一起後，
能完整回答該問題。
請只用純 JSON 格式回應，不要包含任何 Markdown，例如：
{{
    "sub_questions": [
        "舊金山的人口是多少？",
        "舊金山的預算是多少？",
        "舊金山的 GDP 是多少？"
    ]
}}
以下是使用者的問題：{question}
"""

In [ ]:
print(prompt)

In [ ]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(
    model="gpt-5-mini",
    temperature=0,
    json_mode=False
)

In [ ]:
response = llm.complete(prompt)

In [ ]:
response.text

In [ ]:
json.loads(response.text)

In [ ]:
import json

In [ ]:
json.loads(response.text)